In [1]:
import pandas as pd
from google.colab import drive
drive.mount("/content/drive")

xlsx = pd.ExcelFile("/content/drive/MyDrive/Colab Notebooks/DATA 301 Final Project/SSDB_Raw_Data.xlsx")

Mounted at /content/drive


In [2]:
df_legislation = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DATA 301 Final Project/raw_data.csv")

In [3]:
df_incidents = pd.read_excel(xlsx, "INCIDENT")
df_incidents.head()

,Incident_ID,Sources,Number_News,Media_Attention,Reliability,Date,Quarter,School,City,State,...,Accomplice,Hostages,Barricade,Officer_Involved,Bullied,Domestic_Violence,Gang_Related,Preplanned,Shots_Fired,Active_Shooter_FBI
0,20200727AZCAO,https://www.kgun9.com/news/education/man-dies-...,3,Local,4,2020-07-27,Summer,Canyon del Oro High School,Oro Valley,AZ,...,No,No,No,No,No,No,No,No,NaN,No
1,20200721SCBRO,https://abcnews4.com/news/crime-news/person-of...,2,Local,3,2020-07-21,Summer,Brookdale Elementary School,Orangeburg,SC,...,No,No,No,No,No,No,NaN,No,NaN,No
2,20200714MIWID,https://www.detroitnews.com/story/news/local/m...,1,Local,3,2020-07-14,Summer,Willow Woods Elementary School,Detroit,MI,...,No,No,No,No,No,No,No,No,NaN,No
3,20200704INLAM,https://www.abc57.com/news/murder-suspect-char...,2,Local,4,2020-07-04,Summer,Lake Hills Elementary School,Michigan City,IN,...,No,No,No,No,No,No,No,No,NaN,No
4,20200701ILFRP,https://www.dailyherald.com/news/20200714/pros...,1,Local,3,2020-07-01,Summer,Frost Elementary School,Prospect Heights,IL,...,Yes,No,No,No,No,No,Yes,No,NaN,No


In [4]:
df_weapons = pd.read_excel(xlsx, "WEAPON")
df_weapons.head()

,incidentid,weaponcaliber,weapondetails,weapontype
0,19700105DCHIW,Unknown,NaN,Handgun
1,19700105DCSOW,.22 caliber,NaN,Handgun
2,19700105DCUNW,Unknown,NaN,Handgun
3,19700206OHJOC,Unknown,NaN,Handgun
4,19700323CADAL,Unknown,NaN,Handgun


In [5]:
df_weapons.drop(['weaponcaliber', 'weapondetails'], inplace=True, axis=1)
df_weapons.head()

,incidentid,weapontype
0,19700105DCHIW,Handgun
1,19700105DCSOW,Handgun
2,19700105DCUNW,Handgun
3,19700206OHJOC,Handgun
4,19700323CADAL,Handgun


In [6]:
df_weapons.rename(columns={"incidentid": "Incident_ID", "weapontype": "Weapon_Type"}, inplace=True)
df_weapons.head()

,Incident_ID,Weapon_Type
0,19700105DCHIW,Handgun
1,19700105DCSOW,Handgun
2,19700105DCUNW,Handgun
3,19700206OHJOC,Handgun
4,19700323CADAL,Handgun


In [7]:
from google.colab import files
df_weapons.to_csv("weapons.csv", index=False)
files.download("weapons.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
xlsx2 = pd.ExcelFile("/content/drive/MyDrive/Colab Notebooks/DATA 301 Final Project/Pop_Estimates.xlsx")
df_state_pops1 = pd.read_excel(xlsx2)
df_state_pops1.head()

,Fips,Area Name,1969,1970,1971,1972,1973,1974,1975,1976,...,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012
0,0,United States,201298000,203798722,206817509,209274882,211349205,213333635,215456585,217553859,...,290107933,292805298,295516599,298379912,301231207,304093966,306771529,308745538,308745538,308745538
1,1,Alabama,3440000,3449846,3497452,3540080,3580769,3627805,3680533,3737204,...,4503491,4530729,4569805,4628981,4672840,4718206,4757938,4779736,4779736,4779736
2,2,Alaska,296000,304328,316494,326494,333232,344696,370973,393115,...,648414,659286,666946,675302,680300,687455,698895,710231,710231,710231
3,4,Arizona,1737000,1794912,1896108,2008847,2125281,2224342,2286348,2347976,...,5510364,5652404,5839077,6029141,6167681,6280362,6343154,6392017,6392017,6392017
4,5,Arkansas,1913000,1930077,1972028,2018116,2058491,2100385,2158291,2168688,...,2724816,2749686,2781097,2821761,2848650,2874554,2896843,2915918,2915918,2915918


In [9]:
df_state_pops1.drop(0, inplace=True)
df_state_pops1.drop("Fips", inplace=True, axis=1)

In [10]:
df_state_pops1 = pd.melt(df_state_pops1, id_vars=["Area Name"])
df_state_pops1.rename(columns={"Area Name": "State", "variable": "Year", "value": "Population"}, inplace=True)
df_state_pops1.head()

,State,Year,Population
0,Alabama,1969,3440000
1,Alaska,1969,296000
2,Arizona,1969,1737000
3,Arkansas,1969,1913000
4,California,1969,19711000


In [11]:
import requests
import json
all_populations = []
for i in range (2013, 2018):
  response = requests.get("https://datausa.io/api/data?drilldowns=State&measures=Population&year=" + str(i))
  data_populations = response.json()["data"]
  all_populations.extend(data_populations)

In [12]:
from pandas.io.json import json_normalize
df_state_pops2 = json_normalize(all_populations)
df_state_pops2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,ID State,State,ID Year,Year,Population,Slug State
0,04000US01,Alabama,2013,2013,4833722,alabama
1,04000US02,Alaska,2013,2013,735132,alaska
2,04000US04,Arizona,2013,2013,6626624,arizona
3,04000US05,Arkansas,2013,2013,2959373,arkansas
4,04000US06,California,2013,2013,38332521,california


In [13]:
pr = df_state_pops2.index[df_state_pops2["State"] == "Puerto Rico"].tolist()

In [14]:
df_state_pops2.drop(pr, inplace=True)
df_state_pops2.drop(["ID State", "ID Year", "Slug State"], inplace=True, axis=1)

In [15]:
df_state_pops = pd.concat([df_state_pops1, df_state_pops2])
df_state_pops["Year"] = df_state_pops["Year"].astype(int)
df_state_pops = df_state_pops[(df_state_pops["Year"] > 1990) & (df_state_pops["Year"] < 2018)]
df_state_pops.head()

,State,Year,Population
1122,Alabama,1991,4099156
1123,Alaska,1991,570193
1124,Arizona,1991,3788576
1125,Arkansas,1991,2383144
1126,California,1991,30470736


In [16]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC"
}

In [17]:
df_state_pops["State"] = df_state_pops["State"].map(us_state_to_abbrev)
df_state_pops.head()

,State,Year,Population
1122,AL,1991,4099156
1123,AK,1991,570193
1124,AZ,1991,3788576
1125,AR,1991,2383144
1126,CA,1991,30470736


In [18]:
df_state_pops.to_csv("state_pops.csv", index=False)
files.download("state_pops.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
df_incidents[["Year", "Month", "Day"]] = df_incidents["Date"].str.split("-", expand = True)
df_incidents[["Year", "Month", "Day"]] = df_incidents[["Year", "Month", "Day"]].astype(int)
df_incidents_1991_2017 = df_incidents[(df_incidents["Year"] > 1990) & (df_incidents["Year"] < 2018)]
df_incidents_1991_2017.head()

,Incident_ID,Sources,Number_News,Media_Attention,Reliability,Date,Quarter,School,City,State,...,Officer_Involved,Bullied,Domestic_Violence,Gang_Related,Preplanned,Shots_Fired,Active_Shooter_FBI,Year,Month,Day
277,20171231WAPIM,https://www.heraldnet.com/news/police-say-they...,NaN,NaN,2,2017-12-31,Winter,Pinewood Elementary School,Marysville,WA,...,No,No,No,No,No,60,No,2017,12,31
278,20171231LAEDA,https://www.nola.com/crime/index.ssf/2017/01/a...,NaN,NaN,2,2017-12-31,Winter,Edna Karr High School,Algiers,LA,...,No,No,No,No,No,<30,No,2017,12,31
279,20171227CALIL,https://www.dailynews.com/2017/12/28/man-argui...,NaN,NaN,2,2017-12-27,Winter,Lincoln Elementary School,Lancaster,CA,...,No,No,No,No,No,2,No,2017,12,27
280,20171219MIBEB,http://www.wnem.com/story/37105109/breaking-po...,NaN,NaN,2,2017-12-19,Winter,Beecher High School,Beecher,MI,...,No,No,No,No,No,NaN,No,2017,12,19
281,20171214TXELD,https://www.nbcdfw.com/news/local/Dallas-ISD-G...,NaN,NaN,2,2017-12-14,Winter,Elisha M. Pease Elementary School,Dallas,TX,...,No,No,No,No,No,1,No,2017,12,14


In [20]:
df_incidents_1991_2017.to_csv("incidents_1991_2017.csv", index=False)
files.download("incidents_1991_2017.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
df_legislation["state"] = df_legislation["state"].map(us_state_to_abbrev)
df_legislation.head()

,state,year,age18longgunpossess,age18longgunsale,age21handgunpossess,age21handgunsale,age21longgunpossess,age21longgunsale,age21longgunsaled,alcoholism,...,universal,universalh,universalpermit,universalpermith,violent,violenth,violentpartial,waiting,waitingh,lawtotal
0,AL,1991,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,15
1,AK,1991,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
2,AZ,1991,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
3,AR,1991,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15
4,CA,1991,0,1,0,1,0,0,0,1,...,1,1,1,1,1,1,1,1,1,58


In [22]:
df_legislation.to_csv("legislation.csv", index=False)
files.download("legislation.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>